In [9]:
# Check device number
!ls -ltrh /dev/video*

crw-rw---- 1 root video 81, 0 Jul  3 22:05 /dev/video0


In [10]:
import os
import ipywidgets as widgets
from IPython.display import display
import cv2
import random
import uuid

In [11]:
image_path = '../data/shopping-cart/images/cart25.jpg'
image_widget = widgets.Image(value=open(image_path, 'rb').read(), format='jpg')
image_widget.layout.width = '320px'
image_widget.layout.height = '240px'


In [12]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

# Data Augmentation
### Rotate, and Zoom In randomly

In [13]:
# img = cv2.imread('images/cart1.jpg')

def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img
def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img
def zoom(img, value):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, h, w)
    return img

for filename in os.listdir('../data/shopping-cart/images'):
    if filename.endswith('.jpeg') or filename.endswith('.jpg'):
        
        image_path = os.path.join('../data/shopping-cart/images', filename)
        img = cv2.imread(image_path)
        
        img = rotation(img, 30)
        img = zoom(img, 0.5)
        
        # Generate a unique filename
        flnm = str(uuid.uuid4()) + '.jpg'
        # Specify the folder path
        folder_path = '../data/shopping-cart/augmented_data'
        # Combine the folder path and filename
        output_path = os.path.join(folder_path, flnm)
        cv2.imwrite(output_path, img)
        

In [14]:
# display(image_widget)

# Show Unaugmented Data

In [15]:
folder_path = '../data/shopping-cart/images'
image_widgets = []
for filename in os.listdir(folder_path):
    if filename.endswith('.jpeg') or filename.endswith('.jpg'):
        image_path = os.path.join(folder_path, filename)
        image_widget = widgets.Image(value=open(image_path, 'rb').read(), format='jpeg')
        image_widget.layout.width = '320px'
        image_widget.layout.height = '240px'
        image_widgets.append(image_widget)

        
# Create a list to store the HBox containers for each row
row_containers = []

# Iterate over the image widgets and group them into rows
for i in range(0, len(image_widgets), 4):
    row_widgets = image_widgets[i:i+3]
    row_container = widgets.HBox(row_widgets)
    row_containers.append(row_container)

# Display the row containers
for row_container in row_containers:
    display(row_container)        

# Show Augmented Data

In [17]:
folder_path = '../data/shopping-cart/augmented_data'
image_widgets = []
for filename in os.listdir(folder_path):
    if filename.endswith('.jpeg') or filename.endswith('.jpg'):
        image_path = os.path.join(folder_path, filename)
        image_widget = widgets.Image(value=open(image_path, 'rb').read(), format='jpeg')
        image_widget.layout.width = '320px'
        image_widget.layout.height = '240px'
        image_widgets.append(image_widget)

        
# Create a list to store the HBox containers for each row
row_containers = []

# Iterate over the image widgets and group them into rows
for i in range(0, len(image_widgets), 4):
    row_widgets = image_widgets[i:i+3]
    row_container = widgets.HBox(row_widgets)
    row_containers.append(row_container)

# Display the row containers
for row_container in row_containers:
    display(row_container)        
        
        

# Create Dataset

In [20]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

TASK = 'diy'

CATEGORIES = ['shopping_cart', 'no_shopping_cart']

DATASETS = ['A', 'B']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
   datasets[name] = ImageClassificationDataset('../data/classification/' + TASK + '_' + name, CATEGORIES, TRANSFORMS)
    
print("{} task with {} categories defined".format(TASK, CATEGORIES))

# Set up the data directory location if not there already
DATA_DIR = '/nvdli-nano/data/classification/'
!mkdir -p {DATA_DIR}

diy task with ['shopping_cart', 'no_shopping_cart'] categories defined
